# Generate set of administrative regions to use in exposure grid
- The idea here is to get a globally comprehensive set of administrative boundaries at the `adm1` level or higher if the `adm1` level is not available
- Use `adm1` if available and `adm0` if not

INPUTS
- `sset.PATH_GADM`

OUTPUTS
- `sset.PATH_GADM_ADM1`

In [1]:
from sliiders import settings as sset

import pandas as pd
import geopandas as gpd

import warnings

warnings.filterwarnings("ignore", message=".*initial implementation of Parquet.*")

#### Read full `adm0` and `adm1` sets

In [ ]:
adm0 = gpd.read_file(sset.PATH_GADM, layer="level0")
adm1 = gpd.read_file(sset.PATH_GADM, layer="level1")

# Select columns
adm0 = adm0[["GID_0", "geometry"]].copy()
adm1 = adm1[["GID_0", "GID_1", "geometry"]].copy()

#### Exclude non-territory ISOs
- `ATA`: Antarctica
- `XCA`: Caspian Sea

In [4]:
sset.EXCLUDED_ISOS

['ATA', 'XCA']

In [ ]:
# Filter out excluded ISOs
adm0 = adm0[~adm0["GID_0"].isin(sset.EXCLUDED_ISOS)].copy()
adm1 = adm1[~adm1["GID_0"].isin(sset.EXCLUDED_ISOS)].copy()

#### Select only countries in `adm0` that do not appear in the `adm1` set, to be included as intermediate `adm1` regions

In [ ]:
adm0_as_adm1 = adm0[~adm0["GID_0"].isin(adm1["GID_0"].unique())].copy()

#### Concatenate all `adm1` regions

In [ ]:
full = pd.concat([adm0_as_adm1, adm1], ignore_index=True)

#### Assign a comprehensive set of unique IDs
- `GID_1` (`adm1`) for actual `adm1` regions
- `adm0` ID for `adm0_as_adm1` regions

In [ ]:
full["UID"] = full["GID_1"].fillna(full["GID_0"])

### Save to file

In [ ]:
full.to_parquet(sset.PATH_GADM_ADM1, index=False)